## Code violation data etl  
Run the comcate Roll-up Case Detail Report, selecting your date range for 'oped date'.  Select one extra field 'Contacts', and add the phone numbers.  Run the report, selecting 'xls' output. 

NOTE:  This will produce an HTML FRAGMENT file of the form report<sequence numnber>.xls (not sure why, or if it will be the same after the upgrade).

Rename and repair the html in the file. NOTE AM OR PM needed on line 3

> ls report* | sed -e 's/.xls//g' | xargs -I {} mv {}.xls {}.html  
> ls report*.html | xargs -I {} tail -n +20 {} | sed -e 's/.*>Report Details.*<table/<table/g' > rollup.html  
> sed -e 's\/\<p\>Number of Cases.*PM\/\/g' rollup.html > rollupfixed.html  
> sed -e 's^\<\/tr\>^\<\/tr>\<tr\>^g' rollupfixed.html > rollup.html


<span style="color:blue">********* TRY JOINING ON address = " ".join(df[['premsnum','premsnam','premstyp']])</span>

In [28]:
!pip install simpledbf

In [29]:
import re
import pandas as pd
import numpy as np
from simpledbf import Dbf5
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
from os import walk

def get_at(list, index, default=None):
  return list[index] if max(~index, index) < len(list) else default

df = pd.read_html('drive/My Drive/comcate/rollup-20210731.html')

In [32]:
vdf = pd.concat(df)
vdf.APN = vdf.APN.fillna("0").apply(lambda x: x.strip(u'\u200b')).astype(int)

#vdf = vdf[vdf.Module == "Code Enforcement"].drop_duplicates()
print(len(vdf))

369


In [33]:
vdf

,Case#,eFM Case#,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,Zip,APN,Initiation,Open Date,Follow Up Date,Contacts,Assigned To
0,2021-688,NaN,1st Notice of Violation Given,Weeds/Sidewalks - Open,Medium,325,NaN,HENRY,St,NaN,21613.0,1007146787,Neighborhood Patrol,07/30/2021,08/09/2021,Property Owner Muse Property Management LLC 25...,"Wyatt, Sandra"
1,2021-687,NaN,1st Notice of Violation Given,Weeds/Sidewalks - Open,Medium,303,NaN,HENRY,St,NaN,21613.0,1007120222,Neighborhood Patrol,07/30/2021,08/09/2021,Property Owner Henderson Real Estate Investmen...,"Wyatt, Sandra"
2,2021-686,NaN,1st Notice of Violation Given,Weeds/Sidewalks - Open,Medium,112,NaN,TALBOT,Ave,NaN,21613.0,1007109334,Neighborhood Patrol,07/30/2021,08/09/2021,Property Owner KELSEY RENSE HEATH AND SIEFER A...,"Wyatt, Sandra"
3,2021-685,NaN,1st Notice of Violation Given,Weeds/Sidewalks - Open,Medium,302,NaN,MILL,St,NaN,21613.0,1007191936,Neighborhood Patrol,07/30/2021,08/09/2021,Property Owner Fields Rhondana 26550 Arcadia S...,"Wyatt, Sandra"
4,2021-684,NaN,1st Notice of Violation Given,Weeds/Sidewalks - Open,Medium,300,NaN,MILL,St,NaN,21613.0,1007180004,Neighborhood Patrol,07/30/2021,08/09/2021,Property Owner REDEEMER PRESBYTERIAN CHURCH OF...,"Wyatt, Sandra"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,2021-324,NaN,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,901,NaN,LA GRANGE,Ave,NaN,21613.0,1007107323,Neighborhood Patrol,06/02/2021,06/09/2021,Property Owner GENOVA ANTONIA & JOANN10 LONDON...,"Wyatt, Sandra"
365,2021-323,NaN,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,306,NaN,BAYLY,Ave,NaN,21613.0,1007135912,Neighborhood Patrol,06/02/2021,06/09/2021,Property Owner Messer Gabrrielle L Etal Brunec...,"Wyatt, Sandra"
366,2021-322,NaN,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,1009,NaN,WASHINGTON,St,NaN,21613.0,1007144423,Neighborhood Patrol,06/02/2021,06/09/2021,Property Owner DOTY GEORGE F207 LINTHICUM Dr C...,"Wyatt, Sandra"
367,2021-321,NaN,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,716,NaN,PINE,St,NaN,21613.0,1007126492,Neighborhood Patrol,06/02/2021,06/09/2021,Property Owner Farrow Duane 716 Pine St Cambri...,"Wyatt, Sandra"


In [34]:
vdf.query('APN == 0')
# vdf.APN = vdf.APN.astype(float).fillna(0).astype(int)
# vdf[vdf.APN == 1007127049]

,Case#,eFM Case#,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,Zip,APN,Initiation,Open Date,Follow Up Date,Contacts,Assigned To
368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [35]:
vdf = vdf.query('APN != 0')
sdat = pd.read_csv('drive/My Drive/pita 2021/SDAT-CAN-ref-202107.csv')
print("violation data:",len(vdf),"sdat",len(sdat))

violation data: 368 sdat 23212


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,11,12,13,14,15,16,29,38,45,49,52,53,77,82,86,104,106,112,113,129,137) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
def clean_up_columns(df):
  df.columns=df.columns.str.replace('#','')
  return df.drop(columns=[x for x in df.columns if x.startswith('Unnamed')])

def fix_add_str(x):
  if str(x) == "0":
    return ''
  else:
    return str(x).replace('NAN','').replace('nan','').replace(' ','').strip()

vdf_address_fields = ['Street Number','Street Direction','Street Name','Street Type'] # 'Street Direction',
sdat_address_fields = ['strtnum','strtdir','strtnam','strttyp']
sdat_premise_fields = ['premsnum','premsdir','premsnam','premstyp']

vdf = clean_up_columns(vdf)
#vdf.address = vdf.address.apply(lambda x: np.nan if x == "-1" else x)
#vdf[vdf['Case#'] == "2021-4"]
#vdf

In [37]:
# vdf_have_apn_and_address = vdf

In [38]:
#vdf['Street Number'] = vdf['Street Number'].apply(lambda x: x.upper())
vdf['Street Name'] = vdf['Street Name'].apply(lambda x: str(x).upper())
vdf['Street Type'] = vdf['Street Type'].apply(lambda x: str(x).upper())

#sdat = sdat.query('premcity == "CAMBRIDGE"')
sdat_fields = ['strtnum','strtnam','strttyp'] # 'strtdir',
sdat.rename(columns={'acctid':'APN'},inplace=True)
sdat.strtnum = sdat.strtnum.fillna(0.0).astype(float).apply(lambda x: str(int(x)).strip())

vdf.APN = vdf.APN.fillna(0).astype(int)
vdf_have_apn = vdf.query('APN != 0').copy()

vdf_no_apn = vdf.query('APN == 0')

In [39]:
# the number of unique violations prefixes (is that important here?)
len(set([x for x  in sorted(vdf['Violation(s)'].apply(lambda x: str(x).split('-')[0].strip()).unique())]))

17

### create an address column on the violation data

In [40]:
vdf_have_apn.rename(columns={'acctid':'APN'},inplace=True)
sdat_addresses = sdat[['APN','address']].fillna("-1")

vdf_with_sdat_addr = vdf_have_apn.merge(sdat_addresses,on='APN',how='outer',indicator=True).query('_merge == "both"').query('address != "-1"').query('_merge == "both"')
vdf_no_addr = vdf_have_apn.merge(sdat_addresses,on='APN',how='outer',indicator=True).query('_merge == "both"').query('address == "-1"').query('_merge == "both"')
vdf_no_addr2 = vdf_have_apn.merge(sdat_addresses,on='APN',how='outer',indicator=True).query('_merge == "left_only"')#.query('address != "-1"').query('_merge == "left_only"')
#vdf_no_addr['address'] = vdf_no_addr.apply(lambda x: " ".join([fix_add_str(str(x['Street Number'])),fix_add_str(x['Street Direction']),fix_add_str(x['Street Name']),fix_add_str(x['Street Type'])]).strip(),axis=1)
#print("now have sdat address:",len(vdf_with_sdat_addr),"fixed:",len(vdf_no_addr),"total:",len(vdf_with_sdat_addr)+len(vdf_no_addr),len(vdf_have_apn),"of",len(vdf))
#vdf_have_apn_and_address = pd.concat([vdf_with_sdat_addr,vdf_no_addr]).drop(columns='_merge')
vdf_no_addr2.address = vdf_no_addr2.address.fillna("-1")
vdf_no_addr = pd.concat([vdf_no_addr,vdf_no_addr2])

In [41]:
print(len(vdf_with_sdat_addr),len(vdf_no_addr),len(vdf_no_apn),len(vdf_with_sdat_addr)+len(vdf_no_addr)+len(vdf_no_apn))

352 16 0 368


In [43]:
vdf_no_addr
sdat[sdat.APN == 1007219504]

,APN,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,...,crtarcod,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,sdat,google_maps,struct_sqft,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address
20495,1007219504,DORC,480620.5,99758.7,2.401997e+10,2.401997e+11,81,N,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,CAMBRIDGE,NaN,ANCHOR POINT INC,NaN,ANCHOR POINT INC,902 PEACHBLOSSOM AVE,NaN,CAMBRIDGE,MD,21613.0,NaN,NaN,NaN,ROBBINS,ST,CAMBRIDGE,21613.0,NaN,"LOT 9-5,480 SQ. FT.",N/S ROBBINS ST,E/S SPRINGFIELD AVE,NaN,1580.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,https://sdat.dat.maryland.gov/RealProperty/Pag...,MDPV2017_18,NaN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.56168215...,0.0,18100.0,0.0,0.0,NaN,NaN,ROBBINS,ST,CAMBRIDGE,21613.0,NaN


Run all 3 through fixups, then sdat join with premise address  
-- if there's no apn, the best thing to do is join with the manual fix spreadsheet first.  
-- if there is an apn, sdat did not resolve it, and any that don't get fixed by the spreadsheet are in the same category.  
Try to fill them from the sdat 'premXXX' fields.

In [24]:
vdf_with_sdat_addr[vdf_with_sdat_addr.APN == 1007127049]

,Case,eFM Case,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,Zip,APN,Initiation,Open Date,Follow Up Date,Contacts,Assigned To,address,_merge


In [25]:
# get fix what we can with sdat address match <-- this adds extra rows because of Hurlock matches
vdf_fixups = vdf_no_apn.drop(columns=['APN']).merge(sdat.query('city != "HURLOCK"')[sdat_address_fields + ['APN']],left_on=vdf_address_fields,right_on=sdat_address_fields,how='outer',indicator=True ).query('_merge == "both"').drop(columns=['_merge'])
vdf_no_apn = vdf_no_apn.drop(columns=['APN']).merge(sdat.query('city != "HURLOCK"')[sdat_address_fields + ['APN']],left_on=vdf_address_fields,right_on=sdat_address_fields,how='outer',indicator=True ).query('_merge == "left_only"').drop(columns=['_merge'])
# build address with these validated fields
vdf_fixups.APN = vdf_fixups.APN.astype(int)
vdf_fixups['address'] = vdf_fixups.apply(lambda x: " ".join([fix_add_str(str(x['strtnum'])),fix_add_str(x['strtnam']),fix_add_str(x['strttyp'])]).strip(),axis=1)

vdf_have_apn_and_address = pd.concat([vdf_with_sdat_addr,vdf_fixups]).drop(columns=sdat_address_fields)
print("fixed up:",len(vdf_fixups),len(vdf_with_sdat_addr),"-->",len(vdf_have_apn_and_address),len(vdf_no_apn)+len(vdf_no_addr),"to go")
print(len(vdf_have_apn_and_address),len(vdf)-len(vdf_have_apn_and_address))
vdf_have_apn_and_address = vdf_have_apn_and_address.drop(columns=['_merge'])

ValueError: ignored

In [26]:
def check_for_no_address(df):
  nulls = df[df['address'].isnull()]
  df['address'] = df['address'].astype(str)
  errs = df[df['address'] == "-1"]
  print(len(df),len(nulls),len(errs))
  return nulls,errs

_ = check_for_no_address(vdf_have_apn_and_address)

NameError: ignored

### Normalize addresses on the rows that have APNs by setting them to the SDAT value.  Call this 'vdf_have_apn'.

In [27]:
vdf_have_apn_and_address[vdf_have_apn_and_address.APN == 1007127049]

NameError: ignored

### vdf_rem2 contains the rows that could not be fixed automatically

In [ ]:
replacements = ['Street Number','Street Direction','Street Name','Street Type','APN']
hfdf = clean_up_columns(pd.read_csv('drive/My Drive/comcate/manual-fixups-code_violations-20210609 .csv'))

In [ ]:
hfdf['Street Number'] = hfdf['Street Number'].fillna(0).astype(int).astype(str)
hfdf['Street Number'] = hfdf['Street Number'].apply(lambda x: "" if x == "0" else x)
hfdf['address'] = hfdf.apply(lambda x: " ".join([fix_add_str(str(x['Street Number'])),fix_add_str(x['Street Direction']),fix_add_str(x['Street Name']),fix_add_str(x['Street Type'])]).strip(),axis=1)


In [ ]:
#check_for_no_address()
#vdf_no_addr.drop(columns=['address','_merge']).merge(hfdf[['Case','address']], on='Case',how='outer',indicator=True).query('_merge == "both#"')
#vdf_have_apn_and_address.query('Case == "2021-307"')

In [ ]:
manual_fixes = hfdf[replacements + ['Case','address']].merge(vdf_no_apn.drop(columns=replacements + sdat_address_fields + ['APN']), on='Case',how='outer',indicator=True).query('_merge == "both"')
#manfixes = hfdf.drop(columns=replacements).merge(vdf_rem.drop(columns=replacements)['Case#'], on='Case#',how='outer',indicator=True).query('_merge == "both"').drop(columns=['_merge'])
#vdf_have_apn_and_address[vdf_have_apn_and_address['Case'] == '2019-926']
#manfixes[manfixes['Case'] == '2019-926']
#len(vdf_have_apn_and_address)
print(len(vdf_no_apn),len(manual_fixes),"could be repaired from the spreadsheet")
manual_fixes
sdat[sdat.APN == 1007127049]
sdat[sdat.strtnam == "EDGEWOOD"]


75 75 could be repaired from the spreadsheet


,APN,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,...,crtarcod,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,sdat,google_maps,struct_sqft,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address
6040,1007127049,DORC,479636.4,100730.4,2.401997e+10,2.401997e+11,81,N,SF,509 EDGEWOOD AVE,509,NaN,EDGEWOOD,AVE,NaN,NaN,P,CAMBRIDGE,21613.0,GLG HOLDINGS LLC,NaN,GLG HOLDINGS LLC,C/O THOMAS BRADLEY,811 LOCUST ST,CAMBRIDGE,MD,21613.0,NaN,509.0,NaN,EDGEWOOD,AVE,CAMBRIDGE,21613.0,NaN,IMPSL23-25-27-29-31-33 &,"35-17,500 SQ. FT.",EDGEWOOD AVE-CAMBRIDGE,MLB,572.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.57052479...,3200,90333,1964,509,NaN,NaN,EDGEWOOD,AVE,CAMBRIDGE,21613.0,509 EDGEWOOD AVE
6102,1007127820,DORC,479693.0,100739.6,2.401997e+10,2.401997e+11,81,H,SF,504 EDGEWOOD AVE,504,NaN,EDGEWOOD,AVE,NaN,NaN,P,CAMBRIDGE,21613.0,LAKE ALLEN AND TRACY,NaN,LAKE ALLEN AND TRACY,504 EDGEWOOD AVE,NaN,CAMBRIDGE,MD,21613.0,2109.0,504.0,NaN,EDGEWOOD,AVE,CAMBRIDGE,21613.0,NaN,"IMPSL18-20-5,000 SQ. FT.",EDGEWOOD AVE.-ADDITION,CAMBRIDGE,MLB,989.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,A,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.57060256...,1056,118000,2010,504,NaN,NaN,EDGEWOOD,AVE,CAMBRIDGE,21613.0,504 EDGEWOOD AVE
6108,1007127944,DORC,479610.9,100625.9,2.401997e+10,2.401997e+11,81,H,SF,606 EDGEWOOD AVE,606,NaN,EDGEWOOD,AVE,NaN,NaN,P,CAMBRIDGE,21613.0,CORNISH GAVIN K AND SANDRA D,NaN,CORNISH GAVIN K AND SANDR,606 EDGEWOOD AVE,NaN,CAMBRIDGE,MD,21613.0,NaN,606.0,NaN,EDGEWOOD,AVE,CAMBRIDGE,21613.0,NaN,"IMPSL 54&56-5,000 SQ FT",606 EDGEWOOD AVENUE,CAMBRIDGE,MLB,706.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,A,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.56958572...,1056,109967,2005,606,NaN,NaN,EDGEWOOD,AVE,CAMBRIDGE,21613.0,606 EDGEWOOD AVE
7401,1007147562,DORC,479621.0,100640.3,2.401997e+10,2.401997e+11,81,N,SF,602 EDGEWOOD AVE,602,NaN,EDGEWOOD,AVE,NaN,NaN,P,CAMBRIDGE,21613.0,ENNALS JUSTIN C,NaN,ENNALS JUSTIN C,627 WASHINGTON ST,NaN,CAMBRIDGE,MD,21613.0,NaN,602.0,NaN,EDGEWOOD,AVE,CAMBRIDGE,21613.0,NaN,"IMPSL 50&52-5,000 SQ.FT.",602 EDGEWOOD AVENUE,CAMBRIDGE,NaN,1343.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.56971453...,0,2000,0,602,NaN,NaN,EDGEWOOD,AVE,CAMBRIDGE,21613.0,602 EDGEWOOD AVE
8005,1007157134,DORC,479684.7,100727.6,2.401997e+10,2.401997e+11,81,H,SF,506 EDGEWOOD AVE,506,NaN,EDGEWOOD,AVE,NaN,NaN,P,CAMBRIDGE,21613.0,BENSON RENA,NaN,BENSON RENA,506 EDGEWOOD AVE,NaN,CAMBRIDGE,MD,21613.0,2109.0,506.0,NaN,EDGEWOOD,AVE,CAMBRIDGE,21613.0,NaN,"IMPSLOT 22,24-5,000 SQFT",EDGEWOOD AVE.-ADDITION,CAMBRIDGE,MLB,870.0,...,NaN

In [ ]:
#vdf_have_apn_and_address = pd.concat([manfixes.drop(columns=['_merge']).merge(sdat[['APN','address']],on='APN',how='outer',indicator=True).query('_merge == "both"').drop(columns=['_merge']),vdf_have_apn_and_address])
vdf_have_apn_and_address = pd.concat([manual_fixes[vdf_have_apn_and_address.columns],vdf_have_apn_and_address])
vdf_have_apn_and_address.drop_duplicates(inplace=True)
print(len(vdf_have_apn_and_address.drop_duplicates()),len(vdf)-len(vdf_have_apn_and_address))
vdf_have_apn_and_address
_ = check_for_no_address(vdf_have_apn_and_address)
_ = check_for_no_address(vdf_no_addr)
                                     

3515 158
3515 0 0
158 0 158


In [ ]:
#vdf_have_apn_and_address.at[127, 'APN'] = 1007131119
vdf_have_apn_and_address[vdf_have_apn_and_address.APN == 1007127049]
#vdf_no_addr.at[558, 'APN'] = 1007154755
#vdf_have_apn_and_address.at[3651, 'APN'] = 1007158182

#vdf_have_apn_and_address.at[3650, 'address'] = "221 HENRY ST"
#vdf_have_apn_and_address[vdf_have_apn_and_address['Case#'] == "2021-279"]
#vdf_have_apn_and_address[vdf_have_apn_and_address['APN'] < 1005000000]

,Case,eFM Case,Module,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,City,APN,Initiation,Open Date,Follow Up Date,Contacts,Assigned To,address
42,2018-1219,NaN,Code Enforcement,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,517,NaN,EDGEWOOD,AVE,NaN,Cambridge,1007127049,Neighborhood Patrol,08/23/2018,09/03/2018,Property Owner GLG Holdings LLC c/o Thomas Bra...,"Parks, Todd",517 EDGEWOOD AVE
43,2018-1218,NaN,Code Enforcement,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,511,NaN,EDGEWOOD,AVE,NaN,Cambridge,1007127049,Neighborhood Patrol,08/23/2018,09/03/2018,Property Owner GLG Holdings LLC c/o Thomas Bra...,"Parks, Todd",511 EDGEWOOD AVE
44,2018-874,NaN,Code Enforcement,Closed: Voluntary Compliance,"Groups R-2, R-3, R-4, AND I-1 - ClosedDoors - ...",Medium,511,NaN,EDGEWOOD,AVE,NaN,Cambridge,1007127049,Complaint,06/12/2018,07/12/2018,Property Owner GLG Holdings LLC. c/o Thomas Br...,"Parks, Todd",511 EDGEWOOD AVE
143,2021-272,NaN,Code Enforcement,1st Notice of Violation Given,General (Interior) - OpenUnsafe Structures (In...,Medium,NaN,NaN,SQ. FT.,NAN,NaN,CAMBRIDGE,1007127049,Complaint,05/20/2021,06/21/2021,Property Owner GLG HOLDING LLC C/O Thomas Brad...,"Wyatt, Sandra",509 EDGEWOOD AVE
144,2021-271,NaN,Code Enforcement,Complaint Filed,General (Interior) - OpenUnsafe Conditions (In...,Medium,NaN,NaN,SQ. FT.,NAN,NaN,CAMBRIDGE,1007127049,Complaint,05/20/2021,06/21/2021,Property Owner GLG HOLDING LLC C/O Thomas Brad...,"Wyatt, Sandra",509 EDGEWOOD AVE
145,2020-1045,NaN,Code Enforcement,Closed: Voluntary Compliance,Abandoned/Inoperable Vehicle(s) - Closed,Medium,NaN,NaN,SQ. FT.,NAN,NaN,CAMBRIDGE,1007127049,Complaint,08/03/2020,07/18/2020,Property Owner GLG HOLDING LLC C/O THOMAS BRAD...,"Brown, Frank",509 EDGEWOOD AVE
146,2020-836,NaN,Code Enforcement,Closed: Voluntary Compliance,General (Exterior Structure) - ClosedUnsafe St...,Medium,NaN,NaN,SQ. FT.,NAN,NaN,CAMBRIDGE,1007127049,Complaint,07/02/2020,07/31/2020,Property Owner GLG HOLDING LLC C/O THOMAS BRAD...,"Brown, Frank",509 EDGEWOOD AVE
147,2020-835,NaN,Code Enforcement,Closed: Voluntary Compliance,General (Exterior Structure) - ClosedUnsafe St...,Medium,NaN,NaN,SQ. FT.,NAN,NaN,CAMBRIDGE,1007127049,Complaint,07/02/2020,07/31/2020,Property Owner GLG HOLDING LLC C/O THOMAS BRAD...,"Brown, Frank",509 EDGEWOOD AVE
148,2020-834,NaN,Code Enforcement,Closed: Voluntary Compliance,General (Exterior Structure) - ClosedUnsafe St...,Medium,NaN,NaN,SQ. FT.,NAN,NaN,CAMBRIDGE,1007127049,Complaint,07/02/2020,07/31/2020,Property Owner GLG HOLDING LLC C/O THOMAS BRAD...,"Brown, Frank",509 EDGEWOOD AVE
149,2020-833,NaN,Code Enforcement,Closed: Voluntary Compliance,General (Exterior Structure) - ClosedUnsafe St...,Medium,NaN,NaN,SQ. FT.,NAN,NaN,CAMBRIDGE,1007127049,Complaint,07/01/2020,07/30/2020,Property Owner GLG HOLDING LLC C/O THOMAS BRAD...,"Brown, Frank",509 EDGEWOOD AVE


In [ ]:
vdf_no_addr['address'] = vdf_no_addr.apply(lambda x: " ".join([fix_add_str(x['Street Number']),fix_add_str(x['Street Direction']),fix_add_str(x['Street Name']),fix_add_str(x['Street Type'])]).strip(),axis=1)
_ = check_for_no_address(vdf_no_addr)
vdf_have_apn_and_address = pd.concat([vdf_have_apn_and_address,vdf_no_addr[vdf_no_addr['address'] != ""].drop(columns=['_merge'])]).drop_duplicates()
_ = check_for_no_address(vdf_have_apn_and_address)
vdf_no_addr = vdf_no_addr[vdf_no_addr['address'] == ""]
_ = check_for_no_address(vdf_no_addr)
vdf_no_addr
#vdf_have_apn_and_address[vdf_have_apn_and_address.duplicated(subset=['Case#'],keep=False)]

158 0 0
3666 0 0
7 0 0


,Case,eFM Case,Module,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,City,APN,Initiation,Open Date,Follow Up Date,Contacts,Assigned To,address,_merge
53,2021-300,NaN,Code Enforcement,Verbal Warning Given,Abandoned/Inoperable Vehicle(s) - OpenMotor Ve...,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007194668,Complaint,05/26/2021,06/09/2021,Property Owner Mauk Anna M & Mauk Judith M817 ...,"Wyatt, Sandra",,both
2359,2020-697,NaN,Code Enforcement,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007127782,Neighborhood Patrol,05/27/2020,06/03/2020,Property Owner Delmarva Community Services IMC...,"Parks, Todd",,both
2360,2020-695,NaN,Code Enforcement,Closed: Unfounded,Weeds/Sidewalks - Closed,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007127782,Neighborhood Patrol,05/27/2020,06/03/2020,Property Owner Delmarva Comminity Services IMC...,"Parks, Todd",,both
2398,2020-647,NaN,Code Enforcement,Closed: Voluntary Compliance,301.3 Vacant Structures and Land - ClosedAccum...,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007142382,Complaint,05/14/2020,06/14/2020,Property Owner Slaughter Harold Theodore3055 H...,"Parks, Todd",,both
2399,2020-646,NaN,Code Enforcement,Closed: Unfounded,301.3 Vacant Structures and Land - ClosedWeeds...,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007142382,Complaint,05/14/2020,05/21/2020,Property Owner Slaughter Harold Thedore3055 Ha...,"Parks, Todd",,both
558,2021-162,NaN,Code Enforcement,Closed - Error: Written in Error,Weeds/Sidewalks - Closed,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007154755,Neighborhood Patrol,05/05/2021,05/14/2021,Property Owner Madison Bay Holdings LLC1240 Pe...,"Seeberger, Larry",,left_only
1299,2020-1468,NaN,Code Enforcement,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007158182,Neighborhood Patrol,09/23/2020,09/30/2020,Property Owner STRONG& STRONG LLCPO BOX 2408 E...,"Brown, Frank",,left_only


In [ ]:
temp_merge = vdf_no_addr.drop(columns=['_merge','address']).merge(sdat[['APN','premsnum','premsdir','premsnam','premstyp']],on='APN',how='outer',indicator=True,copy=True)
temp_both = temp_merge.query('_merge == "both"')
temp_both['premsnum'] = temp_both.premsnum.fillna(0).astype(int)
temp_both['address'] = temp_both.apply(lambda x: " ".join([fix_add_str(int(x.premsnum)),fix_add_str(x.premsdir),fix_add_str(x.premsnam),fix_add_str(x.premstyp)]).strip(),axis=1)
#_ = check_for_no_address(temp_both)
last_item = temp_merge.query('_merge == "left_only"').drop(columns=['_merge'])#.merge(sdat[['APN','address']],on='APN',how='inner')
temp_both
vdf_have_apn_and_address = pd.concat([vdf_have_apn_and_address,temp_both]).drop_duplicates()
_ = check_for_no_address(vdf_have_apn_and_address)


3672 0 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
last_item
last_item.at[6, 'address'] = "221 HENRY ST"
last_item
# test = pd.concat([vdf_have_apn_and_address,manfixes.drop(columns=['_merge']).merge(sdat[['APN','address']],on='APN',how='outer',indicator=True).query('_merge == "left_only"').drop_duplicates().drop(columns=['_merge'])]).drop_duplicates()
# last2 = test[test.duplicated(subset=['Case#'],keep=False)].query('APN < 1015000000')
#vdf_have_apn_and_address = vdf_have_apn_and_address.drop(columns=['Open Date','Follow Up Date']).merge(vdf[['Case#','Open Date','Follow Up Date']],on='Case#',how='inner')
#manfixes.merge(sdat[['APN','address']],on='APN',how='outer',indicator=True).query('_merge == "left_only"')

,Case,eFM Case,Module,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,City,APN,Initiation,Open Date,Follow Up Date,Contacts,Assigned To,premsnum,premsdir,premsnam,premstyp,address
6,2020-1468,NaN,Code Enforcement,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007158182,Neighborhood Patrol,09/23/2020,09/30/2020,Property Owner STRONG& STRONG LLCPO BOX 2408 E...,"Brown, Frank",NaN,NaN,NaN,NaN,221 HENRY ST


In [ ]:
vdf_have_apn_and_address = pd.concat([vdf_have_apn_and_address,last_item])[list(vdf.columns)+ ['address']]

In [ ]:
_ = check_for_no_address(vdf_have_apn_and_address)

3515 0 0


In [ ]:
vdf_have_apn_and_address = vdf_have_apn_and_address.drop(columns=['Open Date','Follow Up Date']).merge(vdf[['Case#','Open Date','Follow Up Date']], on='Case#',how='inner')

In [ ]:
vdf_have_apn_and_address.to_csv('drive/My Drive/comcate/code_violations-20210801.csv')

In [ ]:
vdf_have_apn_and_address[~vdf_have_apn_and_address['Open Date'].str.contains('/')]

,Case#,eFM Case#,Module,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,City,APN,Initiation,Contacts,Assigned To,address,_merge,Open Date,Follow Up Date


In [ ]:
vdf_have_apn_and_address = clean_up_columns(vdf_have_apn_and_address)
vdf_have_apn_and_address = vdf_have_apn_and_address[vdf_have_apn_and_address['APN'] != 1015008458]
vdf_have_apn_and_address.APN = vdf_have_apn_and_address.APN.astype(int)
print(len(vdf_have_apn_and_address))

vdf_have_apn_and_address['address'] = vdf_have_apn_and_address['address'].astype(str)
vdf_have_apn_and_address[vdf_have_apn_and_address['address'] == "-1"]
#vdf_have_apn_and_address.to_csv('drive/My Drive/comcate/code_violations-20210628.csv')
#results2#.to_csv('drive/My Drive/comcate/code_violations-20210609.csv')

3673


,Case#,eFM Case#,Module,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,City,APN,Initiation,Contacts,Assigned To,address,_merge,Open Date,Follow Up Date
104,2021-307,NaN,Code Enforcement,1st Notice of Violation Given,"Weeds/Sidewalks - OpenWeed, Grass and Vegetati...",Medium,NaN,NaN,W/S CEDAR,ST,NaN,CAMBRIDGE,1007126336,Neighborhood Patrol,Property Owner DEEP HARBOUR LLC8423 Veterans H...,"Wyatt, Sandra",-1,NaN,05/27/2021,06/10/2021
128,2021-300,NaN,Code Enforcement,Verbal Warning Given,Abandoned/Inoperable Vehicle(s) - OpenMotor Ve...,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007194668,Complaint,Property Owner Mauk Anna M & Mauk Judith M817 ...,"Wyatt, Sandra",-1,NaN,05/26/2021,06/09/2021
201,2021-281,NaN,Code Enforcement,Sent For Abatement,Weeds/Sidewalks - Open,Medium,NaN,NaN,S/S LEONARD'S,NAN,NaN,CAMBRIDGE,1007160100,Neighborhood Patrol,Property Owner CHESAPEAKE UTILITIES CORPORATIO...,"Wyatt, Sandra",-1,NaN,05/25/2021,06/01/2021
325,2021-242,NaN,Code Enforcement,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,NaN,NaN,N/S CEDAR,ST,NaN,CAMBRIDGE,1007178735,Neighborhood Patrol,Property Owner WHEATLEY JAMES DAVID JR205 CEDA...,"Wyatt, Sandra",-1,NaN,05/18/2021,05/25/2021
326,2021-241,NaN,Code Enforcement,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,NaN,NaN,N/S CEDAR,ST,NaN,CAMBRIDGE,1007128169,Neighborhood Patrol,Property Owner Wheatley James David Jr5882 Ric...,"Wyatt, Sandra",-1,NaN,05/18/2021,05/25/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257,2019-565,NaN,Code Enforcement,Closed: Unfounded,Infestation - Closed,Medium,301-303-305-,NaN,GAY,ST,NaN,CAMBRIDGE,1007174470,Witness Call-In,Property Owner DORCHESTER COUNTY PUBLIC LIBRAR...,"Parks, Todd",-1,NaN,08/13/2019,08/13/2019
3408,2019-228,NaN,Code Enforcement,Closed: Voluntary Compliance,"Weed, Grass and Vegetation - ClosedSanitation ...",Medium,NaN,NaN,BETHEL CEMETERY-BAYLY,RD,NaN,CAMBRIDGE,1007175280,Self-Initiated,Property Owner UNION BAPTIST MISSIONARY SOCIET...,"Webb,, Susan",-1,NaN,04/23/2019,05/28/2019
3409,2019-221,NaN,Code Enforcement,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,NaN,NaN,S/S MD RT 16,NAN,NaN,CAMBRIDGE,1007144008,Neighborhood Patrol,Property Owner Jerome Travers5605 Belle Aire R...,"Parks, Todd",-1,NaN,04/22/2019,04/29/2019
3534,2018-1369,NaN,Code Enforcement,Closed: Voluntary Compliance,Accumulation of Rubbish/Garbage - ClosedAbando...,Medium,NaN,NaN,S/FACTORY C,NAN,NaN,CAMBRIDGE,1007168829,Self-Initiated,Property Owner KOSKI FAMILY LIMITED PARTNERSHI...,"Webb,, Susan",-1,NaN,09/27/2018,02/15/2019


In [ ]:
#vdf_have_apn_and_address[vdf_have_apn_and_address['APN'] < 1000000000]#[vdf_have_apn_and_address.APN ==1007178379]
sdat.query('ownname1.str.contains("STRONG")',engine='python')
#vdf_have_apn_and_address.drop(columns=['_merge','strtnum','strtnam','strttyp'])

,APN,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,...,crtarcod,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,sdat,google_maps,struct_sqft,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address
4802,1007105371,DORC,481706.9,100407.6,2.401997e+10,2.401997e+11,81,H,SF,709 MARYLAND AVE,709,NaN,MARYLAND,AVE,NaN,NaN,P,CAMBRIDGE,21613.0,ARMSTRONG DAVID B,NaN,ARMSTRONG DAVID B,709 MARYLAND AVE,NaN,CAMBRIDGE,MD,21613.0,2013.0,709.0,NaN,MARYLAND,AVE,CAMBRIDGE,21613.0,NaN,"IMPSLOT 7,500 SQ. FT.",709 MARYLAND AVE,CAMBRIDGE,MLB,384.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,A,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.56742777...,1056,89233,1935,709,NaN,NaN,MARYLAND,AVE,CAMBRIDGE,21613.0,709 MARYLAND AVE
5634,1007120915,DORC,479847.3,99201.9,2.401997e+10,2.401997e+11,81,N,SF,1313 COLONIAL AVE,1313,NaN,COLONIAL,AVE,NaN,NaN,P,CAMBRIDGE,21613.0,STRONG & STRONG LLC,NaN,STRONG & STRONG LLC,PO BOX 2408,NaN,EASTON,MD,21601.0,8947.0,1313.0,NaN,COLONIAL,AVE,CAMBRIDGE,21613.0,NaN,"IMPSLOT 5,400 SQ. FT.",1313 COLONIAL AVENUE,CAMBRIDGE,MLB,688.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.55673657...,768,65300,1958,1313,NaN,NaN,COLONIAL,AVE,CAMBRIDGE,21613.0,1313 COLONIAL AVE
7231,1007144849,DORC,479868.8,100122.2,2.401997e+10,2.401997e+11,81,N,SF,702 DOUGLAS ST,702,NaN,DOUGLAS,ST,NaN,NaN,P,CAMBRIDGE,21613.0,STRONG & STRONG,A MARYLAND LIMITED LIABILITY COMPA,STRONG & STRONG,PO BOX 2408,NaN,EASTON,MD,21601.0,NaN,702.0,NaN,DOUGLAS,ST,CAMBRIDGE,21613.0,NaN,"IMPSLOT 48-2,205 SQ.FT.",702 DOUGLAS STREET,CAMBRIDGE,NaN,1191.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.56502496...,510,19700,1952,702,NaN,NaN,DOUGLAS,ST,CAMBRIDGE,21613.0,702 DOUGLAS ST
9394,1007178530,DORC,479809.7,99211.3,2.401997e+10,2.401997e+11,81,N,SF,1310 RAINBOW AVE,1310,NaN,RAINBOW,AVE,NaN,NaN,P,CAMBRIDGE,21613.0,STRONG & STRONG LLC,NaN,STRONG & STRONG LLC,PO BOX 2408,NaN,EASTON,MD,21601.0,8947.0,1310.0,NaN,RAINBOW,AVE,CAMBRIDGE,21613.0,NaN,IMPSP/O 46 & 47-48-49,"11,220 SF-RAINBOW AVE.",CAMBRIDGE,MLB,688.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.55682465...,1056,125833,2011,1310,NaN,NaN,RAINBOW,AVE,CAMBRIDGE,21613.0,1310 RAINBOW AVE
20379,1007158181,DORC,479443.5,100160.8,2.401997e+10,2.401997e+11,81,N,NaN,827 WASHINGTON ST,0,NaN,NaN,NaN,NaN,NaN,NaN,CAMBRIDGE,21613.0,STRONG & STRONG LLC,NaN,STRONG & STRONG LLC,PO BOX 2408,NaN,EASTON,MD,21601.0,NaN,827.0,NaN,WASHINGTON,ST,CAMBRIDGE,21613.0,NaN,"IMPVLOT 4,921 SQ. FT.",N/S WASH ST-E/S RIGBY AV,CAMBRIDGE,NaN,1559.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
vdf[vdf['Violation(s)'].str.match(r'(^4-.*)')==True]

,Case#,eFM Case#,Module,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,City,APN,Initiation,Open Date,Follow Up Date,Contacts,Assigned To
2109,2019-863,NaN,Code Enforcement,Closed: Voluntary Compliance,4-44.5 Bamboo - Closed,Medium,312,NaN,GLENBURN,AVE,NaN,CAMBRIDGE,1007109741,Complaint,10/07/2019,11/06/2019,Property Owner BRIAN & AUDREY WOZNY312 GLENBUR...,"Webb,, Susan"
2111,2019-861,NaN,Code Enforcement,Closed: Voluntary Compliance,4-44.5 Bamboo - Closed,Medium,310,NaN,GLENBURN,AVE,NaN,CAMBRIDGE,1007121792,Complaint,10/04/2019,12/06/2019,Property Owner GRAY DONALD W AND GRAY MARY L31...,"Webb,, Susan"


In [ ]:
len(results)

NameError: ignored